# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [33]:
# your code here
import pandas as pd
ttt = pd.read_csv('tic-tac-toe.csv')

In [34]:
ttt

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,x,x,x,x,o,o,x,o,o,True
1,x,x,x,x,o,o,o,x,o,True
2,x,x,x,x,o,o,o,o,x,True
3,x,x,x,x,o,o,o,b,b,True
4,x,x,x,x,o,o,b,o,b,True
...,...,...,...,...,...,...,...,...,...,...
953,o,x,x,x,o,o,o,x,x,False
954,o,x,o,x,x,o,x,o,x,False
955,o,x,o,x,o,x,x,o,x,False
956,o,x,o,o,x,x,x,o,x,False


In [35]:
# true: 1 false: 0
# b = 0, o=1, x=2

def naughts2nums(x):
    if isinstance(x, bool):
        if x:
            return 1
        else:
            return 0
    else:
        if x == 'x':
            return 1
        elif x == 'o':
            return -1
        else:
            return 0
           

In [36]:
ttt['class'] = ttt['class'].apply(naughts2nums)
for c in ttt.columns[:9]:
    ttt[c] = ttt[c].apply(naughts2nums)
ttt

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
0,1,1,1,1,-1,-1,1,-1,-1,1
1,1,1,1,1,-1,-1,-1,1,-1,1
2,1,1,1,1,-1,-1,-1,-1,1,1
3,1,1,1,1,-1,-1,-1,0,0,1
4,1,1,1,1,-1,-1,0,-1,0,1
...,...,...,...,...,...,...,...,...,...,...
953,-1,1,1,1,-1,-1,-1,1,1,0
954,-1,1,-1,1,1,-1,1,-1,1,0
955,-1,1,-1,1,-1,1,1,-1,1,0
956,-1,1,-1,-1,1,1,1,-1,1,0


In [39]:
ttt_output = ttt[['class']]
ttt_input = ttt.drop(['class'],axis=1)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [42]:
# your code here
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ttt_input, ttt_output)

from sklearn.datasets import fetch_openml
from sklearn.exceptions import ConvergenceWarning
from sklearn.neural_network import MLPClassifier

nn = MLPClassifier(hidden_layer_sizes= (15,15), solver='sgd', verbose=10, max_iter=200, random_state=1)

In [66]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# perhaps you need to do
#!pip install keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils.np_utils import to_categorical

model = Sequential([ # as far as we know, all networks are sequential
  Dense(81, activation='relu', input_shape=(9,)), # 784= 28*28 dense networks means all neurons in one layer are connected to all neuronsof the next layer
  Dense(36, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
  #Dense(18, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
  Dense(9, activation='relu'),      # choosing relu instead of sigmoid, this is somewhat common
  Dense(2, activation='softmax'),   # the softmax actiavation is the last one to compensate for the high volume additions
])

# Compile the model.
model.compile(
  optimizer='adam', #here we could use stochastic gradient descent, but adam is a de facto standard
  loss='categorical_crossentropy', #this is how we create the original blame to play the blame game
  metrics=['accuracy'],
)

# Train the model.
history = model.fit(
  X_train,
  to_categorical(y_train), # just to make sure the outputs are not considered numeric (because, ya know, they are numbers...)
  epochs=30, # go 5 times through the whole dataset
  batch_size=32, # send 32 images at a time before you tweak the network again, to make it faster
)

Epoch 1/30
23/23 [==============================] - 0s 1ms/step - loss: 0.6884 - accuracy: 0.5557
Epoch 2/30
23/23 [==============================] - 0s 2ms/step - loss: 0.6331 - accuracy: 0.7326
Epoch 3/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5731 - accuracy: 0.7646
Epoch 4/30
23/23 [==============================] - 0s 2ms/step - loss: 0.5155 - accuracy: 0.7813
Epoch 5/30
23/23 [==============================] - 0s 1ms/step - loss: 0.4644 - accuracy: 0.8050
Epoch 6/30
23/23 [==============================] - 0s 1ms/step - loss: 0.4125 - accuracy: 0.8412
Epoch 7/30
23/23 [==============================] - 0s 1ms/step - loss: 0.3556 - accuracy: 0.8830
Epoch 8/30
23/23 [==============================] - 0s 2ms/step - loss: 0.2971 - accuracy: 0.9262
Epoch 9/30
23/23 [==============================] - 0s 3ms/step - loss: 0.2371 - accuracy: 0.9554
Epoch 10/30
23/23 [==============================] - 0s 3ms/step - loss: 0.1715 - accuracy: 0.9735
Epoch 11/30
23/23 [

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [67]:
# your code here
# Evaluate the model.
model.evaluate(
  X_test,
  to_categorical(y_test)
)

8/8 [==============================] - 0s 1000us/step - loss: 0.0194 - accuracy: 0.9958


[0.019437814131379128, 0.9958333373069763]

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here

**Which approach(es) did you find helpful to improve your model performance?**

# your answer here
I started off with density of 9,9,2, increasing this helped.

adding more layers and tweaking the numbers a bit allowed us to reach an accuracy of 1.000 and a loss of 0.0000801 in training, though the test accuracy reamained the same at .9917 / 0.0133

In [68]:
removing one of the layers I added, still kept a high training accuracy of 1 though with more loss, still low that at 0.0019 - this did perform a bit better though on the test: .9958 / 0.0194

SyntaxError: invalid syntax (Temp/ipykernel_6884/257389551.py, line 1)